In [26]:
import numpy as np
import pandas as pd
import random

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

In [ ]:
df_ratings = pd.read_csv("ml-latest-small/ratings.csv")
df_pivot = df_ratings.pivot(index="userId", columns="movieId", values="rating")

corr_matrix = df_pivot.T.corr(method="pearson")
mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
df_pearson = (
    corr_matrix.where(mask)
    .stack()
    .rename_axis(["user_1", "user_2"])
    .reset_index(name="pearson_corr")
)

user_avg = df_pivot.mean(axis=1)
df_user_avg_filled = df_pivot.apply(lambda row: row.fillna(user_avg[row.name]), axis=1)
chosen_df_filled = df_user_avg_filled
user_similarity = cosine_similarity(chosen_df_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=chosen_df_filled.index, columns=chosen_df_filled.index)

In [28]:
k = 40
df_top_k_users = (
    df_pearson
    .groupby('user_1', group_keys=True)
    .apply(lambda grp: grp.nlargest(k, 'pearson_corr'))
    .reset_index(drop=True)
)

def get_top_k_neighbors(u, similarity_df, k=10):
    # Prende la riga corrispondente all'utente u (similarità con tutti gli altri)
    sim_row = similarity_df.loc[u].drop(u, errors='ignore')  # escludi l'utente stesso
    # Ordina in base alla similarità decrescente e prendi i primi k
    top_k = sim_row.nlargest(k).index
    return top_k

C:\Users\lollo\AppData\Local\Temp\ipykernel_8828\491380758.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.nlargest(k, 'pearson_corr'))


In [29]:
def train_regression_for_user(u, df_pivot, user_avg, neighbors, min_common=2, verbose=False):
    """
    Ritorna (w_{vu}, mse) dove:
      - w_{vu} è il vettore dei pesi per i 'neighbors' di utente u,
      - mse è l'errore quadratico medio sul "training set" di u.
      
    La regressione risolve:
        (r_{uj} - mu_u) = sum_v [ w_{vu} * (r_{vj} - mu_v) ]
    per tutti gli item j che u ha effettivamente valutato.
    """
    # Lista di item valutati da u
    user_ratings = df_pivot.loc[u]
    items_rated_by_u = user_ratings[user_ratings.notna()].index

    # Vettore target Y e matrice X
    X = []
    Y = []

    mu_u = user_avg[u]  # media dell'utente u

    for item_j in items_rated_by_u:
        r_uj = df_pivot.loc[u, item_j]  # rating di u su item_j
        y_j = r_uj - mu_u               # (r_{uj} - mu_u)
        
        row_features = []
        # Costruisco i valori (r_{vj} - mu_v) per ciascun vicino v
        for v in neighbors:
            r_vj = df_pivot.loc[v, item_j]
            if pd.isna(r_vj):
                # Se il vicino non ha valutato item_j, uso 0 (oppure la media di v come fallback)
                row_features.append(0.0)
            else:
                mu_v = user_avg[v]
                row_features.append(r_vj - mu_v)

        X.append(row_features)
        Y.append(y_j)

    # Converto in array numpy
    X = np.array(X)
    Y = np.array(Y)

    # Se ho meno di min_common item per allenare, restituisco pesi nulli e mse=0 o np.nan
    if len(Y) < min_common:
        if verbose:
            print(f"[User {u}] Pochi item per la regressione: {len(Y)} < {min_common}")
        return np.zeros(len(neighbors)), np.nan

    # Alleno un modello di regressione lineare senza intercetta
    model = LinearRegression(fit_intercept=False)
    model.fit(X, Y)

    # Calcolo il MSE sul training set
    Y_pred = model.predict(X)
    mse = np.mean((Y_pred - Y)**2)

    if verbose:
        print(f"[User {u}] MSE: {mse:.4f}")

    return model.coef_, mse


In [30]:
user_regression_models = {}
all_mse = []

for u in df_pivot.index:
    # Trova i k vicini di u
    neighbors_u = get_top_k_neighbors(u, user_similarity_df, k)
    
    # Allena la regressione per utente u
    w_u, mse_u = train_regression_for_user(
        u=u,
        df_pivot=df_pivot,
        user_avg=user_avg,
        neighbors=neighbors_u,
        min_common=2,
        verbose=True  # <-- per stampare il MSE
    )
    
    # Salva i risultati
    user_regression_models[u] = {
        'neighbors': neighbors_u,
        'weights': w_u
    }
    all_mse.append(mse_u)

# Stampa un MSE globale medio (escludendo eventuali NaN)
all_mse = [m for m in all_mse if not np.isnan(m)]
if all_mse:
    print(f"\nMSE medio su tutti gli utenti: {np.mean(all_mse):.4f}")
else:
    print("Nessun MSE disponibile (troppi utenti con pochi rating?).")


[User 1] MSE: 0.5672
[User 2] MSE: 0.4354
[User 3] MSE: 3.7782
[User 4] MSE: 1.3868
[User 5] MSE: 0.3183
[User 6] MSE: 0.5746
[User 7] MSE: 1.2942
[User 8] MSE: 0.3173
[User 9] MSE: 1.2287
[User 10] MSE: 0.8882
[User 11] MSE: 0.4709
[User 12] MSE: 0.3625
[User 13] MSE: 0.5280
[User 14] MSE: 0.2541
[User 15] MSE: 0.8670
[User 16] MSE: 0.1903
[User 17] MSE: 0.1886
[User 18] MSE: 0.3344
[User 19] MSE: 0.7391
[User 20] MSE: 1.1709
[User 21] MSE: 0.9139
[User 22] MSE: 1.6898
[User 23] MSE: 0.4262
[User 24] MSE: 0.1811
[User 25] MSE: 0.0356
[User 26] MSE: 0.1652
[User 27] MSE: 0.8849
[User 28] MSE: 0.5667
[User 29] MSE: 0.3167
[User 30] MSE: 0.0278
[User 31] MSE: 0.3172
[User 32] MSE: 1.2251
[User 33] MSE: 8.5289
[User 34] MSE: 1.0463
[User 35] MSE: 0.1404
[User 36] MSE: 0.7206
[User 37] MSE: 0.1467
[User 38] MSE: 0.4911
[User 39] MSE: 0.8245
[User 40] MSE: 0.7364
[User 41] MSE: 0.9752
[User 42] MSE: 0.9740
[User 43] MSE: 0.3673
[User 44] MSE: 0.9199
[User 45] MSE: 0.8298
[User 46] MSE: 0.28

In [31]:
def predict_rating_regression(u, j, df_pivot, user_avg, user_regression_models):
    """
    Restituisce la predizione di r_{uj} usando i pesi appresi dal modello di regressione.
    """
    # Se l'utente non è nel dizionario, rating sconosciuto
    if u not in user_regression_models:
        return np.nan

    neighbors = user_regression_models[u]['neighbors']
    weights = user_regression_models[u]['weights']
    mu_u = user_avg[u]

    # Costruisco la somma dei contributi
    contribution_sum = 0.0
    for w_vu, v in zip(weights, neighbors):
        r_vj = df_pivot.loc[v, j]
        if pd.isna(r_vj):
            r_vj = user_avg[v]  # Se manca il rating, potresti usare la media di v come fallback
        mu_v = user_avg[v]
        contribution_sum += w_vu * (r_vj - mu_v)

    return mu_u + contribution_sum


In [134]:
chosen_user = random.choice(df_pivot.index)
chosen_item = random.choice(df_pivot.columns)

pred = predict_rating_regression(chosen_user, chosen_item, df_pivot, user_avg, user_regression_models)
print(f"Rating previsto per l'utente {chosen_user} sull'item {chosen_item}: {pred:.2f}")


Rating previsto per l'utente 283 sull'item 3802: 3.31


In [135]:
max = 0
for i in df_pivot.columns:
    pred = predict_rating_regression(266, i, df_pivot, user_avg, user_regression_models)
    if pred > max:
        max = pred
        item = i
print(f"Rating previsto per l'utente 266 sull'item {item}: {max:.2f}")

Rating previsto per l'utente 266 sull'item 58559: 135.85
